In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from utils.load_dataframe import load_dataframe

In [ ]:
# orders = load_dataframe('orders')
# reviews = load_dataframe('order_reviews')
# products = load_dataframe('products')

# display(orders.dtypes)
# display(reviews.dtypes)
# display(products.dtypes)


# Com a função load_dataframe fica de forma automática os tipos de cada coluna, e agora não precisa ficar convertendo a cada novo arquivo criado. Basta chama-la que as colunas ficarão com os tipos corretos.

In [ ]:
# Evolução de pedidos ao longo do tempo

orders = load_dataframe("orders")
order_items = load_dataframe("order_items")
products = load_dataframe("products")
customers = load_dataframe("customer")
sellers = load_dataframe("sellers")

In [ ]:
df = orders.merge(order_items, on='order_id')
df = df.merge(products[['product_id', 'product_category_name']], on='product_id')
df = df.merge(customers[['customer_id', 'customer_state']], on='customer_id')
df = df.merge(sellers[['seller_id', 'seller_state']], on='seller_id')

df['revenue'] = df['price'] * df['order_item_id'] / df['order_item_id']

df.head()

In [ ]:
orders_by_month = orders.groupby(orders['order_purchase_timestamp'].dt.to_period('M')).size()

plt.figure(figsize=(12,6))
plt.plot(orders_by_month.index.to_timestamp(), orders_by_month.values, marker='o')
plt.title("Evolução de Pedidos por Mês")
plt.xlabel("Mês")
plt.ylabel("Número de Pedidos")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M')
revenue_by_month = df.groupby('order_month')['revenue'].sum()

ticket_by_month = df.groupby('order_month').agg(ticket=('revenue', 'sum'), pedidos=('order_id', 'nunique'))
ticket_by_month['ticket_medio'] = ticket_by_month['ticket'] / ticket_by_month['pedidos']

plt.figure(figsize=(12,6))
plt.plot(revenue_by_month.index.to_timestamp(), revenue_by_month.values, marker='o', label='Faturamento')
plt.title("Faturamento Mensal")
plt.xlabel("Mês")
plt.ylabel("Faturamento (R$)")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

plt.figure(figsize=(12,6))
plt.plot(ticket_by_month.index.to_timestamp(), ticket_by_month['ticket_medio'], marker='o', color='orange')
plt.title("Ticket Médio Mensal")
plt.xlabel("Mês")
plt.ylabel("Ticket Médio (R$)")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
top_categories = df.groupby('product_category_name', observed=True)['revenue'].sum().sort_values(ascending=False).head(10)


plt.figure(figsize=(10,6))
plt.barh(top_categories.index[::-1], top_categories.values[::-1], color='skyblue')
plt.title("Top 10 Categorias por Faturamento")
plt.xlabel("Faturamento (R$)")
plt.show()


top_sellers = df.groupby('seller_state')['revenue'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
plt.barh(top_sellers.index[::-1], top_sellers.values[::-1], color='lightgreen')
plt.title("Top 10 Sellers por Faturamento")
plt.xlabel("Faturamento (R$)")
plt.show()
